In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

In [10]:
# documents = combination between patterns and intents
print (len(documents), "documents")

47 documents


In [11]:
# classes = intents
print (len(classes), "classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [12]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [13]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [14]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [18]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
47/47 [==============================] - 2s 44ms/step - loss: 2.1957 - accuracy: 0.1277
Epoch 2/200
47/47 [==============================] - 0s 851us/step - loss: 2.1269 - accuracy: 0.2979
Epoch 3/200
47/47 [==============================] - 0s 851us/step - loss: 2.0832 - accuracy: 0.2128
Epoch 4/200
47/47 [==============================] - 0s 1ms/step - loss: 1.9687 - accuracy: 0.4468
Epoch 5/200
47/47 [==============================] - 0s 1ms/step - loss: 1.8481 - accuracy: 0.4043
Epoch 6/200
47/47 [==============================] - 0s 851us/step - loss: 1.6607 - accuracy: 0.5745
Epoch 7/200
47/47 [==============================] - 0s 745us/step - loss: 1.6398 - accuracy: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 745us/step - loss: 1.4617 - accuracy: 0.5957
Epoch 9/200
47/47 [==============================] - 0s 1ms/step - loss: 1.3687 - accuracy: 0.6383
Epoch 10/200
47/47 [==============================] - 0s 851us/step - loss: 1.1741 - accuracy: 0.

47/47 [==============================] - 0s 745us/step - loss: 0.0919 - accuracy: 0.9787
Epoch 80/200
47/47 [==============================] - 0s 638us/step - loss: 0.0308 - accuracy: 1.0000
Epoch 81/200
47/47 [==============================] - 0s 745us/step - loss: 0.0257 - accuracy: 1.0000
Epoch 82/200
47/47 [==============================] - 0s 745us/step - loss: 0.0694 - accuracy: 1.0000
Epoch 83/200
47/47 [==============================] - 0s 851us/step - loss: 0.0252 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 745us/step - loss: 0.0314 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 638us/step - loss: 0.0332 - accuracy: 1.0000
Epoch 86/200
47/47 [==============================] - 0s 745us/step - loss: 0.0336 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 745us/step - loss: 0.0201 - accuracy: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 638us/step - loss: 0.0942 - accuracy: 

47/47 [==============================] - 0s 638us/step - loss: 0.0405 - accuracy: 1.0000
Epoch 159/200
47/47 [==============================] - 0s 532us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 160/200
47/47 [==============================] - 0s 745us/step - loss: 0.0148 - accuracy: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 638us/step - loss: 0.0080 - accuracy: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 745us/step - loss: 0.0027 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 745us/step - loss: 0.0900 - accuracy: 0.9787
Epoch 164/200
47/47 [==============================] - 0s 745us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 638us/step - loss: 0.0141 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 745us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 532us/step - loss: 0.0079 - a

In [19]:
print("model created")

model created
